### doc_topics
ESTABELECE RELAÇÕES ENTRE DOCUMENTOS E TÓPICOS NA BASE DE DADOS NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.

### Observações

#### Sobre como cria tabela CSV com limite documentos por tópico

In [37]:
import nltk
import os
import codecs
import string
import matplotlib.pyplot as plt
import matplotlib as mpl
from gensim import corpora, models, similarities #Latent Dirichlet Allocation implementation with Gensim
import pyLDAvis
import pyLDAvis.gensim
from IPython.display import clear_output

import getpass
from sshtunnel import SSHTunnelForwarder
import pymysql

import pickle

In [38]:
outputs = "../outputs/"

In [39]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Verifica qual sistema operacional está sendo usado. Renato = Linux ; Marcelo = nt (Windows)
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
if os.name == 'nt':
    encoding_type = 'utf-8'
    ssh_priv_key = 'C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa'
    ssh_user='marcelobribeiro'
    sql_user='marcelobribeiro'
else:
    encoding_type = 'ISO-8859-1'
    ssh_priv_key = '/home/rsouza/.ssh/id_rsa'
    ssh_user='rsouza'
    sql_user='rsouza'

#### Carrega o treinamento

In [40]:
#filename = 'C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/outputs/trainedLDA.sav'
file_corpus = '../outputs/LDAcorpus.pkl'
file_dictionary = '../outputs/LDAdictionary.pkl'
corpus = pickle.load(open(file_corpus, 'rb'))
dictionary = pickle.load(open(file_dictionary, 'rb'))

In [41]:
len(corpus)

200

In [42]:
%time lda = models.LdaModel(corpus, num_topics=20, id2word=dictionary, passes=10)

Wall time: 9.59 s


In [43]:
topics_list = lda.print_topics(-1, num_words=10)
topics_list_file = '../outputs/topics_list.pkl'
file_dictionary = '../outputs/LDAdictionary-test.pkl'
pickle.dump(topics_list, open(topics_list_file, 'wb'))
#load_test = pickle.load(open(topics_list_file, 'rb'))

In [44]:
#lda.show_topics(num_topics=-1)

In [45]:
topics_matrix = lda.show_topics(formatted=False, num_words=20, num_topics=-1)
topics_matrix[14]

(14,
 [('silveira', 0.010859238264488541),
  ('embaixador', 0.0097449805801555246),
  ('antonio', 0.005465982275474934),
  ('minuta', 0.004378371087874278),
  ('telegrama', 0.0043721641467850212),
  ('paulo', 0.0043455903834368368),
  ('novo', 0.003396915106125289),
  ('palavras', 0.0033555325854904097),
  ('azeredo', 0.0033259052988623586),
  ('nome', 0.0033078569674308829),
  ('carta', 0.0033045458329930073),
  ('brasilia', 0.0032811025717487288),
  ('excelência', 0.0032806483009904398),
  ('bem', 0.0032793389136955954),
  ('gratissimo', 0.003278938851351327),
  ('autorizo', 0.0032759180114932253),
  ('sinceras', 0.0032720084848344051),
  ('data', 0.0032712375289883893),
  ('cairo', 0.0032706426481891311),
  ('social', 0.0032701648686879097)])

#### Visualiza os tópicos com o PyLDAvis

In [46]:
# https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
7      16.325686        1       1 -0.187825 -0.056254
6       9.182604        1       2 -0.059767 -0.024478
1       8.319022        1       3 -0.101672 -0.066722
15      7.067400        1       4 -0.079713  0.060938
3       7.020220        1       5 -0.001812 -0.007769
18      5.887430        1       6 -0.017798  0.184119
5       5.036761        1       7 -0.005578  0.018615
11      4.798637        1       8 -0.008433  0.000983
12      4.610434        1       9  0.111968 -0.038330
17      4.472043        1      10  0.000858  0.003669
13      4.278557        1      11  0.054345 -0.074937
14      4.063014        1      12  0.027350  0.059765
2       3.335889        1      13 -0.002163 -0.001943
19      2.952043        1      14 -0.020078  0.018555
0       2.643054        1      15  0.018276 -0.009148
9       2.532616        1      16  0.044752 -0.026462
4       2.032956        1      17  0.049344 -0.024445
16      1.943506        1      18  0.058813 -0.009206
8       1.816476        1      19  0.067132 -0.001629
10      1.681654        1      20  0.052000 -0.005322, topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
5013  Default   52.000000        amigo   52.000000  30.0000  30.0000
4035  Default   94.000000          rio   94.000000  29.0000  29.0000
5399  Default   56.000000           gb   56.000000  28.0000  28.0000
287   Default   22.000000      prezado   22.000000  27.0000  27.0000
344   Default   37.000000           lt   37.000000  26.0000  26.0000
4488  Default   54.000000          bsb   54.000000  25.0000  25.0000
3139  Default   89.000000      antonio   89.000000  24.0000  24.0000
3346  Default   20.000000    generosas   20.000000  23.0000  23.0000
6488  Default   76.000000   embaixador   76.000000  22.0000  22.0000
3231  Default   32.000000     agradeço   32.000000  21.0000  21.0000
3     Default   34.000000         nome   34.000000  20.0000  20.0000
5528  Default   25.000000         erro   25.000000  19.0000  19.0000
1067  Default   21.000000          rua   21.000000  18.0000  18.0000
4138  Default   58.000000       vieira   58.000000  17.0000  17.0000
5618  Default   11.000000     tlxgrama   11.000000  16.0000  16.0000
6946  Default   17.000000        paulo   17.000000  15.0000  15.0000
6687  Default   19.000000       buenos   19.000000  14.0000  14.0000
4616  Default   40.000000     exterior   40.000000  13.0000  13.0000
1940  Default   19.000000   excelência   19.000000  12.0000  12.0000
5198  Default   17.000000     parabens   17.000000  11.0000  11.0000
4428  Default   35.000000           df   35.000000  10.0000  10.0000
1304  Default   14.000000   presidente   14.000000   9.0000   9.0000
2618  Default   18.000000   entregatlg   18.000000   8.0000   8.0000
5144  Default   32.000000     afetuoso   32.000000   7.0000   7.0000
4094  Default   35.000000          rjo   35.000000   6.0000   6.0000
6373  Default   26.000000       abraco   26.000000   5.0000   5.0000
1997  Default  113.000000      azeredo  113.000000   4.0000   4.0000
3869  Default   27.000000     ministro   27.000000   3.0000   3.0000
4321  Default   25.000000       minuta   25.000000   2.0000   2.0000
2299  Default   30.000000       241gtx   30.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
2859  Topic20    0.790279    silveirik    2.010841   3.1515  -5.6775
1451  Topic20    0.792595      498/491    7.834875   1.7944  -5.6745
1589  Topic20    0.793883          www   17.067844   1.0174  -5.6729
5528  Topic20    0.791791         erro   25.846030   0.5998  -5.6755
3869  Topic20    0.792450     ministro   27.291781   0.5462  -5.6747
5641  Topic20    0.798406        souto   54.509733  -0.1381  -5.6672
4138  Topic20    0.793890       vieira   58.544321  -0.2152  -5.6729
5799  Top

In [47]:
data_ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output.html'))

#### Gera a distribuição de tópicos para um novo documento
Alimenta a base mysql com dados de score (relação) entre tópicos e documentos, gerando a distribuição de tópicos para um novo documento.

Notas:
* Os documentos são selecionados da base SQL com alguns filtros: idioma português e legibilidade acima de 40%.
* 'pt' = língua portuguesa. 
* No caso de documentos que não conseguimos identificar a legibilidade, por serem curtos (menos de 10 frases totais), foram classificados com a numeração '-1'.

In [53]:
count = 0
pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key = ssh_priv_key,
                        ssh_private_key_password = pass_sshkey,
                        ssh_username=ssh_user,
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user=sql_user,
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    insere dados na base mysql
    captura documentos da tabela docs, transfere para a tabela topic-doc e insere dados de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    cur.execute("DROP TABLE IF EXISTS topic_doc")
    cur.execute('''CREATE TABLE IF NOT EXISTS topic_doc
               (doc_id VARCHAR(31), topic_id smallint(6), topic_score FLOAT
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')

    cur.execute("SELECT * FROM CPDOC_AS.docs WHERE main_language = 'pt' AND (readability > 0.4 OR readability = -1) ")  
    data = cur.fetchall()
    numrows = cur.rowcount
    percentil = numrows/100
    
    for row in data:
        count += 1
        if row is None: break
        
        ### mede percentual de conclusão da tarefa ###
        if count % percentil == 0: 
            clear_output()
            print(int(count/percentil),'% done')
        
        text =  row[4]
        text = text.split()
        text_bow = dictionary.doc2bow(text)
        score_list = lda[text_bow]
        doc_id = row[0]
        for score in score_list:
            topic_id = str(score[0])
            topic_score = str(score[1])
            query = "INSERT INTO topic_doc VALUES (%s, %s, %s)"
            cur.execute(query, (doc_id, topic_id, topic_score))
        #print(doc_id, topic_id, topic_score)   
        cur.execute("ALTER TABLE CPDOC_AS.topic_doc ORDER BY topic_id ASC, topic_score DESC")
        


········
········
